In [1]:
import oci
import logging
import json
from operator import itemgetter
import pprint
import random
import string
import time
from tqdm import tqdm

%load_ext lab_black

In [2]:
config = oci.config.from_file(profile_name="WOLFCOMPARTMENT")
tenancy_ocid = config["tenancy"]
logging.basicConfig()
logging.getLogger("oci").setLevel(logging.ERROR)
with open("compartment_heirarchy.json", "r") as stream:
    compartment_heirarchy = json.load(stream)

In [3]:
def print_compartment_creation_start(compartment_name):
    print("--------------------------------------------------------------------")
    print("STARTED CREATING COMPARTMENT: {}".format(compartment_name))
    print("--------------------------------------------------------------------")


def print_compartment_creation_info(compartment):
    print("--------------------------------------------------------------------")
    print("FINISHED CREATING COMPARTMENT: {}".format(compartment["name"]))
    print("--------------------------------------------------------------------")
    print("COMPPARTMENT DETAILS")
    print("-----------------------------")
    print("compartment_name:  {}".format(compartment["name"]))
    print("compartment_description:  {}".format(compartment["description"]))
    print("compartment_ocid: {}".format(compartment["id"]))
    print("parent_compartment_ocid: {}".format(compartment["compartment_id"]))
    print()


def print_compartment_deletion_info(compartment):
    print("--------------------------------------------------------------------")
    print("STARTED DELETING COMPARTMENT: {}".format(compartment["name"]))
    print("--------------------------------------------------------------------")
    print("COMPARTMENT DETAILS")
    print("-----------------------------")
    print("compartment_name:  {}".format(compartment["name"]))
    print("compartment_description:  {}".format(compartment["description"]))
    print("compartment_ocid: {}".format(compartment["id"]))
    print("parent_compartment_ocid: {}".format(compartment["compartment_id"]))
    print()


def print_heirarchy_statistics(heirarchy):
    compartment_names = extract_value_by_field(heirarchy, "name")
    compartment_descriptions = extract_value_by_field(heirarchy, "description")
    compartments_kv = dict(zip(compartment_names, compartment_descriptions))
    formatted_heirarchy = pprint.pformat(heirarchy, indent=4)
    print("INPUT FILE")
    print("--------------------------------")
    print(formatted_heirarchy)
    print()
    print("LIST OF COMPARTMENTS")
    print("------------------------------- ")
    for compartment_name, compartment_description in compartments_kv.items():
        print("COMPARTMENT NAME: {}".format(compartment_name))
        print("COMPARTMENT DESCRIPTION: {}".format(compartment_description))
        print()
    print("--------------------------------")
    print("TOTAL NUMBER OF COMPARTMENTS: {}".format(len(compartments_kv)))


def print_existing_compartment_decorator():
    print("----------------------------")
    print("Compartment already Exists")
    print("----------------------------")
    print("Skipping Compartment Creation")


def convert_response_to_dict(oci_response):
    return oci.util.to_dict(oci_response.data)

In [4]:
def extract_value_by_field(obj, key):
    """Pull all values of specified key from nested JSON."""
    arr = []

    def extract(obj, arr, key):
        """Recursively search for values of key in JSON tree."""
        if isinstance(obj, dict):
            for k, v in obj.items():
                if isinstance(v, (dict, list)):
                    extract(v, arr, key)
                elif k == key:
                    arr.append(v)
        elif isinstance(obj, list):
            for item in obj:
                extract(item, arr, key)
        return arr

    results = extract(obj, arr, key)
    return results

In [5]:
def wait_until_compartment_is_active(client, compartment_id):
    retries = 5
    for i in tqdm(range(retries)):
        try:
            get_compartment_response = oci.wait_until(
                client,
                client.get_compartment(
                    compartment_id, retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
                ),
                "lifecycle_state",
                "ACTIVE",
            )
        except oci.exceptions.ServiceError as e:
            if i < retries - 1:  # i is zero indexed
                print("Tries " + str(i) + "/5")
                time.sleep(10)
                continue
            else:
                raise
        break
    return True


def create_compartment(
    client,
    parent_compartment_ocid,
    compartment_name,
    description="No Description",
    defined_tags=None,
    freeform_tags=None,
):
    # Check if Current Compartment Exists
    if not checkByName_if_compartment_exists(compartment_name, client):
        # Create the Compartment Details
        print_compartment_creation_start(compartment_name)
        compartment_details = oci.identity.models.CreateCompartmentDetails(
            compartment_id=parent_compartment_ocid,
            name=compartment_name,
            description=description,
        )
        # Create compartment and Wait for State
        response = client.create_compartment(
            create_compartment_details=compartment_details,
            retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        compartment = convert_response_to_dict(response)
        if wait_until_compartment_is_active(
            client=client, compartment_id=compartment["id"]
        ):
            print_compartment_creation_info(compartment=compartment)
            return compartment
    else:
        print_existing_compartment_decorator()
        compartment = fetchCompartment_by_name(
            compartment_name=compartment_name, client=client
        )
        print_compartment_creation_info(compartment)
        return compartment

In [6]:
def checkByName_if_compartment_exists(compartment_name, client):
    compartments = fetch_all_compartments_in_tenancy(client)
    activeCompartments = filter_compartments_by_state(
        compartmentList=compartments, compartmentState="ACTIVE"
    )
    compartmentNames = extract_value_by_field(activeCompartments, "name")
    if compartment_name in compartmentNames:
        return True
    else:
        return False


def checkByOCID_if_compartment_active(compartment_id, client):
    compartment_response = client.get_compartment(
        compartment_id=compartment_id, retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
    )
    compartment = convert_response_to_dict(compartment_response)
    return True if compartment["lifecycle_state"] == "ACTIVE" else False


def fetchCompartment_by_name(compartment_name, client):
    compartments = fetch_all_compartments_in_tenancy(client)
    activeCompartments = filter_compartments_by_state(
        compartmentList=compartments, compartmentState="ACTIVE"
    )
    for compartment in activeCompartments:
        if compartment_name == compartment["name"]:
            return compartment
    print("Compartment {} not found ".format(compartment_name))
    return None


def filter_compartments_by_state(compartmentList=[], compartmentState="ACTIVE"):
    """Filter Compartments by their lifecycle state, ACTIVE| DELETNG | DELETED | CREATING"""
    filteredCompartments = [
        compartment
        for compartment in compartmentList
        if compartment["lifecycle_state"] == compartmentState
    ]
    return filteredCompartments


def fetch_all_compartments_in_tenancy(client):
    """Fetch all Compartments in Tenancy , and look across all subtrees."""
    compartmentResponse = oci.pagination.list_call_get_all_results(
        client.list_compartments,
        compartment_id=tenancy_ocid,
        limit=200,
        access_level="ACCESSIBLE",
        compartment_id_in_subtree=True,
        retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
    )
    return convert_response_to_dict(compartmentResponse)

In [7]:
def create_compartment_heirarchy(heirarchy, client):
    """Recursive Function Implementation to call functions"""

    def validate_inputDict_createCompartment(heirarchy, client, parent):
        if "name" in heirarchy.keys() and "description" in heirarchy.keys():
            compartment = create_compartment(
                client=client,
                parent_compartment_ocid=parent,
                compartment_name=heirarchy["name"],
                description=heirarchy["description"],
            )
            parent = compartment["id"]
        return parent

    def recurse_compartment_heirarchy(heirarchy, client, parent):
        if "compartments" in heirarchy.keys():
            compartment_tree_traversal(
                heirarchy["compartments"], parent, client,
            )

    def compartment_tree_traversal(
        input_compartment_heirarchy, parent_compartment_ocid, client
    ):
        if isinstance(input_compartment_heirarchy, dict):
            parent_compartment_ocid = validate_inputDict_createCompartment(
                heirarchy=input_compartment_heirarchy,
                client=client,
                parent=parent_compartment_ocid,
            )
            recurse_compartment_heirarchy(
                heirarchy=input_compartment_heirarchy,
                client=client,
                parent=parent_compartment_ocid,
            )

        elif isinstance(input_compartment_heirarchy, list):
            for item in tqdm(input_compartment_heirarchy):
                if wait_until_compartment_is_active(client, parent_compartment_ocid):
                    compartment_tree_traversal(item, parent_compartment_ocid, client)

    compartment_tree_traversal(
        input_compartment_heirarchy=heirarchy,
        parent_compartment_ocid=tenancy_ocid,
        client=client,
    )

In [8]:
def verify_ChildCompartment_deletionState(client, compartment_id):
    ## FETCH ALL CHILD COMPARTMENTS FIRST
    compartmentResponse = client.list_compartments(compartment_id=compartment_id)
    compartmentList = convert_response_to_dict(compartmentResponse)
    ## IF THERE ARE ANY CHILD COMPARTMENTS UNDER DELETION, WAIT UNTIL IT IS FINISHED
    deletingCompartments = filter_compartments_by_state(
        compartmentList=compartmentList, compartmentState="DELETING"
    )
    if deletingCompartments:
        for compartment in deletingCompartments:
            new_compartment_id = compartment["id"]
            oci.wait_until(
                client,
                client.get_compartment(
                    compartment_id=new_compartment_id,
                    retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
                ),
                "lifecycle_state",
                "DELETED",
            )


def recurse_And_Delete_Active_ChildCompartments(client, compartment_id):
    ## FETCH ALL CHILD COMPARTMENTS FIRST
    compartmentResponse = client.list_compartments(compartment_id=compartment_id)
    compartmentList = convert_response_to_dict(compartmentResponse)
    ## IF THERE ARE ANY ACTIVE CHILD COMPARTMENTS , RECURSE FURTHER
    activeCompartments = filter_compartments_by_state(
        compartmentList=compartmentList, compartmentState="ACTIVE"
    )
    if activeCompartments:
        for compartment in activeCompartments:
            new_compartment_id = compartment["id"]
            deletedCompartment = delete_compartment_heirarchy(
                compartment_id=new_compartment_id, client=client
            )


def mark_CurrentCompartment_For_Deletion(client, compartment_id):
    ### CHECK IF CURRENT COMPARTMENT MARKED FOR DELETION IS ACTIVE
    if checkByOCID_if_compartment_active(compartment_id=compartment_id, client=client):
        deletedCompartment = client.delete_compartment(
            compartment_id=compartment_id,
            retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )

        deletedCompartment = oci.wait_until(
            client,
            client.get_compartment(
                compartment_id=compartment_id,
                retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
            ),
            "lifecycle_state",
            "DELETING",
        )
        deletedCompartment = convert_response_to_dict(deletedCompartment)
        print_compartment_deletion_info(deletedCompartment)
    else:
        deletedCompartment = client.get_compartment(
            compartment_id=compartment_id,
            retry_strategy=oci.retry.DEFAULT_RETRY_STRATEGY,
        )
        deletedCompartment = convert_response_to_dict(deletedCompartment)

    return deletedCompartment

In [9]:
def delete_compartment_heirarchy(client, compartment_id):
    recurse_And_Delete_Active_ChildCompartments(
        client=client, compartment_id=compartment_id
    )
    verify_ChildCompartment_deletionState(client=client, compartment_id=compartment_id)
    deletedCompartment = mark_CurrentCompartment_For_Deletion(
        client=client, compartment_id=compartment_id
    )
    return deletedCompartment

In [10]:
identity_client = oci.identity.IdentityClient(config)
print_heirarchy_statistics(compartment_heirarchy)
create_compartment_heirarchy(compartment_heirarchy, identity_client)

  0%|          | 0/5 [00:00<?, ?it/s]

INPUT FILE
--------------------------------
{   'compartments': [   {   'compartments': [   {   'compartments': [   {   'description': 'desc-c11',
                                                                            'name': 'compartment-111'},
                                                                        {   'compartments': [   {   'description': 'desc-c1111',
                                                                                                    'name': 'compartment-1121'},
                                                                                                {   'description': 'desc-c11',
                                                                                                    'name': 'compartment-1122'}],
                                                                            'description': 'desc-c112',
                                                                            'name': 'compartment-112'}],
                         

  0%|          | 0/5 [00:01<?, ?it/s]


----------------------------
Compartment already Exists
----------------------------
Skipping Compartment Creation



  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

...

--------------------------------------------------------------------
FINISHED CREATING COMPARTMENT: compartment-21
--------------------------------------------------------------------
COMPPARTMENT DETAILS
-----------------------------
compartment_name:  compartment-21
compartment_description:  desc-c21
compartment_ocid: ocid1.compartment.oc1..aaaaaaaawujncz5ffnbmzjis6q6vlpcsnvd7tvrowkhsldcqo4x2a5dsghha
parent_compartment_ocid: ocid1.compartment.oc1..aaaaaaaadrwwfloagoeovywpzoadtmjiomkovrbxchupgks4tgmr5bvri4ua



  0%|          | 0/5 [00:00<?, ?it/s]


----------------------------
Compartment already Exists
----------------------------
Skipping Compartment Creation



100%|██████████| 2/2 [00:18<00:00,  9.04s/it]

--------------------------------------------------------------------
FINISHED CREATING COMPARTMENT: compartment-22
--------------------------------------------------------------------
COMPPARTMENT DETAILS
-----------------------------
compartment_name:  compartment-22
compartment_description:  desc-c22
compartment_ocid: ocid1.compartment.oc1..aaaaaaaan3tcjlkch2qpv5fl74yc2vefxy66cvnyihu3zyvodkqllqtbpmfq
parent_compartment_ocid: ocid1.compartment.oc1..aaaaaaaadrwwfloagoeovywpzoadtmjiomkovrbxchupgks4tgmr5bvri4ua



In [ ]:
identity_client = oci.identity.IdentityClient(config)
delete_compartment_heirarchy(identity_client, tenancy_ocid)